# Лабораторная работа №5
## ПРИМЕНЕНИЕ СИНГУЛЯРНОГО СПЕКТРАЛЬНОГО АНАЛИЗА ДЛЯ ДЕКОМПОЗИЦИИ ВРЕМЕННОГО РЯДА
### Комлева Юлия Владимировна
### Вариант: 9
### Группа: РИМ-181226

In [2]:
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt
import h5py
%matplotlib inline

Метод сингулярного спектрального анализа SSA реализуется в 2 этапа – разложение и группировка.
### Этап разложения

In [7]:
def SSA_modes(F, L):
    N = len(F)
    K = N - L + 1
    X = np.empty((L, K))
    for i in range(0, L):
        for j in range(0, K):
            X[i,j] = F[i+j]
    
    S = np.dot(X, X.T)
    U, A, _ = np.linalg.svd(S)
    V = np.dot(X.T, U)
    return A, U, V

In [8]:
ts = np.array([3, 2, 1, 2, 3, 2, 1, 2, 3, 2, 1, 2, 3]) # мини временной ряд
A, U, V = SSA_modes(ts, 3) # его разложение с длиной окна = 3
print(A) # собственные числа
print(U) # собственные вектора
print(V) # траекторные вектора

[129.66842566  12.           3.33157434]
[[-5.78869570e-01  7.07106781e-01  4.06091149e-01]
 [-5.74299610e-01  4.14039445e-16 -8.18645196e-01]
 [-5.78869570e-01 -7.07106781e-01  4.06091149e-01]]
[[-3.46407750e+00  1.41421356e+00 -1.29257973e-02]
 [-2.88977789e+00  0.00000000e+00  8.05719399e-01]
 [-3.46407750e+00 -1.41421356e+00 -1.29257973e-02]
 [-4.03837711e+00  8.88178420e-16 -8.31570994e-01]
 [-3.46407750e+00  1.41421356e+00 -1.29257973e-02]
 [-2.88977789e+00  0.00000000e+00  8.05719399e-01]
 [-3.46407750e+00 -1.41421356e+00 -1.29257973e-02]
 [-4.03837711e+00  8.88178420e-16 -8.31570994e-01]
 [-3.46407750e+00  1.41421356e+00 -1.29257973e-02]
 [-2.88977789e+00  0.00000000e+00  8.05719399e-01]
 [-3.46407750e+00 -1.41421356e+00 -1.29257973e-02]]


In [13]:
check_A = np.array([129.66842566, 12., 3.33157434])
check_U = np.array( [[-5.78869570e-01, 7.07106781e-01, 4.06091149e-01],
[-5.74299610e-01, 4.14039445e-16, -8.18645196e-01],
[-5.78869570e-01, -7.07106781e-01, 4.06091149e-01]])
check_V = np.array([[-3.46407750e+00, 1.41421356e+00, -1.29257973e-02],
[-2.88977789e+00, 0.00000000e+00, 8.05719399e-01],
[-3.46407750e+00, -1.41421356e+00, -1.29257973e-02],
[-4.03837711e+00, 8.88178420e-16, -8.31570994e-01],
[-3.46407750e+00, 1.41421356e+00, -1.29257973e-02],
[-2.88977789e+00, 0.00000000e+00, 8.05719399e-01],
[-3.46407750e+00, -1.41421356e+00, -1.29257973e-02],
[-4.03837711e+00, 8.88178420e-16, -8.31570994e-01],
[-3.46407750e+00, 1.41421356e+00, -1.29257973e-02],
[-2.88977789e+00, 0.00000000e+00, 8.05719399e-01],
[-3.46407750e+00, -1.41421356e+00, -1.29257973e-02]])

In [23]:
np.allclose(A, check_A) and np.allclose(U, check_U) and np.allclose(V, check_V)

True

In [80]:
def SSA_group(A, U, V, N, I):
    
    L = len(A)
    K = N - L + 1
    
    
    #шаг группировки
    V = V.transpose()
    Z = np.dot(U[:, I], V[I, :])
    print(Z)
    print("N ", N)
    #этап диагонального усреднения
    G = np.zeros(N)
    L_ = min(L, K)
    K_ = max(L, K)
    for k in range(0, N):
        print("k ", k)
        if 0 <= k < L_ -1:
            G[k] = 1/(k+1)* sum([Z[i][k-i+1] for i in range(k)])
            #G[k] = sum([Z[i][k-i] for i in range(k)])
        if L_-1 <= k < K_:
            G[k] = 1/L_* sum([Z[i][k-i+1] for i in range(L_-1)])
            #G[k] = sum([Z[i][k-i] for i in range(L_-1)])
        if K_ <= k < N+1:
            G[k] = 1/(N-k)* sum([Z[i][k-i+1] for i in range(k-K_+1,N-K_)])
            #G[k] =  sum([Z[i][k-i] for i in range(k-K_+1,N-K_)])
    return G

In [81]:
ts

array([3, 2, 1, 2, 3, 2, 1, 2, 3, 2, 1, 2, 3])

In [82]:
A

array([129.66842566,  12.        ,   3.33157434])

In [83]:
ts1 = SSA_group(A, U, V, len(ts), [0, 1, 2])
print(ts1)
np.allclose(ts, ts1)

[[3. 2. 1. 2. 3. 2. 1. 2. 3. 2. 1.]
 [2. 1. 2. 3. 2. 1. 2. 3. 2. 1. 2.]
 [1. 2. 3. 2. 1. 2. 3. 2. 1. 2. 3.]]
N  13
k  0
k  1
k  2
k  3
k  4
k  5
k  6
k  7
k  8
k  9
k  10


IndexError: index 11 is out of bounds for axis 0 with size 11

In [24]:
k = 4
if 5 > k >=0:
    print("wtf")

wtf
